# Activity: Build an XGBoost model<a href="#Activity:-Build-an-XGBoost-model" class="anchor-link">¶</a>

## Introduction<a href="#Introduction" class="anchor-link">¶</a>

In this activity, you’ll build on the skills and techniques you learned
in the decision tree and random forest lessons to construct your own
XGBoost classification model. The XGBoost model is a very powerful
extension of decision trees, so having a strong working familiarity with
this process will strengthen your skills and resume as a data
professional.

This activity is a continuation of the airlines project in which you
built decision tree and random forest models. You will use the same
data, but this time you will train, tune, and evaluate an XGBoost model.
You’ll then compare the performance of all three models and decide which
model is best. Finally, you’ll explore the feature importances of your
model and identify the features that most contribute to customer
satisfaction.

## Step 1: Imports<a href="#Step-1:-Imports" class="anchor-link">¶</a>

### Import packages<a href="#Import-packages" class="anchor-link">¶</a>

Begin with your import statements. First, import `pandas`, `numpy`, and
`matplotlib` for data preparation. Next, import scikit-learn (`sklearn`)
for model preparation and evaluation. Then, import `xgboost`, which
provides the classification algorithm you'll implement to formulate your
predictive model.

In \[1\]:

    # Import relevant libraries and modules.
    import numpy as np
    import pandas as pd
    import matplotlib as plt
    import pickle

    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    from sklearn import metrics

    from xgboost import XGBClassifier
    from xgboost import plot_importance

### Load the dataset<a href="#Load-the-dataset" class="anchor-link">¶</a>

To formulate your model, `pandas` is used to import a csv of airline
passenger satisfaction data called `Invistico_Airline.csv`. This
DataFrame is called `airline_data`. As shown in this cell, the dataset
has been automatically loaded in for you. You do not need to download
the .csv file, or provide more code, in order to access the dataset and
proceed with this lab. Please continue with this activity by completing
the following instructions.

In \[2\]:

    # RUN THIS CELL TO IMPORT YOUR DATA. 
    airline_data = pd.read_csv('Invistico_Airline.csv', error_bad_lines=False)

### Display the data<a href="#Display-the-data" class="anchor-link">¶</a>

Examine the first 10 rows of data to familiarize yourself with the
dataset.

In \[3\]:

    # Display the first ten rows of data.
    airline_data.head(10)

Out\[3\]:

|     | satisfaction | Customer Type  | Age | Type of Travel  | Class    | Flight Distance | Seat comfort | Departure/Arrival time convenient | Food and drink | Gate location | ... | Online support | Ease of Online booking | On-board service | Leg room service | Baggage handling | Checkin service | Cleanliness | Online boarding | Departure Delay in Minutes | Arrival Delay in Minutes |
|-----|--------------|----------------|-----|-----------------|----------|-----------------|--------------|-----------------------------------|----------------|---------------|-----|----------------|------------------------|------------------|------------------|------------------|-----------------|-------------|-----------------|----------------------------|--------------------------|
| 0   | satisfied    | Loyal Customer | 65  | Personal Travel | Eco      | 265             | 0            | 0                                 | 0              | 2             | ... | 2              | 3                      | 3                | 0                | 3                | 5               | 3           | 2               | 0                          | 0.0                      |
| 1   | satisfied    | Loyal Customer | 47  | Personal Travel | Business | 2464            | 0            | 0                                 | 0              | 3             | ... | 2              | 3                      | 4                | 4                | 4                | 2               | 3           | 2               | 310                        | 305.0                    |
| 2   | satisfied    | Loyal Customer | 15  | Personal Travel | Eco      | 2138            | 0            | 0                                 | 0              | 3             | ... | 2              | 2                      | 3                | 3                | 4                | 4               | 4           | 2               | 0                          | 0.0                      |
| 3   | satisfied    | Loyal Customer | 60  | Personal Travel | Eco      | 623             | 0            | 0                                 | 0              | 3             | ... | 3              | 1                      | 1                | 0                | 1                | 4               | 1           | 3               | 0                          | 0.0                      |
| 4   | satisfied    | Loyal Customer | 70  | Personal Travel | Eco      | 354             | 0            | 0                                 | 0              | 3             | ... | 4              | 2                      | 2                | 0                | 2                | 4               | 2           | 5               | 0                          | 0.0                      |
| 5   | satisfied    | Loyal Customer | 30  | Personal Travel | Eco      | 1894            | 0            | 0                                 | 0              | 3             | ... | 2              | 2                      | 5                | 4                | 5                | 5               | 4           | 2               | 0                          | 0.0                      |
| 6   | satisfied    | Loyal Customer | 66  | Personal Travel | Eco      | 227             | 0            | 0                                 | 0              | 3             | ... | 5              | 5                      | 5                | 0                | 5                | 5               | 5           | 3               | 17                         | 15.0                     |
| 7   | satisfied    | Loyal Customer | 10  | Personal Travel | Eco      | 1812            | 0            | 0                                 | 0              | 3             | ... | 2              | 2                      | 3                | 3                | 4                | 5               | 4           | 2               | 0                          | 0.0                      |
| 8   | satisfied    | Loyal Customer | 56  | Personal Travel | Business | 73              | 0            | 0                                 | 0              | 3             | ... | 5              | 4                      | 4                | 0                | 1                | 5               | 4           | 4               | 0                          | 0.0                      |
| 9   | satisfied    | Loyal Customer | 22  | Personal Travel | Eco      | 1556            | 0            | 0                                 | 0              | 3             | ... | 2              | 2                      | 2                | 4                | 5                | 3               | 4           | 2               | 30                         | 26.0                     |

10 rows × 22 columns

### Display the data type for each column<a href="#Display-the-data-type-for-each-column"
class="anchor-link">¶</a>

Next, observe the types of data present within this dataset.

In \[4\]:

    # Display the data type for each column in your DataFrame.
    airline_data.dtypes

Out\[4\]:

    satisfaction                          object
    Customer Type                         object
    Age                                    int64
    Type of Travel                        object
    Class                                 object
    Flight Distance                        int64
    Seat comfort                           int64
    Departure/Arrival time convenient      int64
    Food and drink                         int64
    Gate location                          int64
    Inflight wifi service                  int64
    Inflight entertainment                 int64
    Online support                         int64
    Ease of Online booking                 int64
    On-board service                       int64
    Leg room service                       int64
    Baggage handling                       int64
    Checkin service                        int64
    Cleanliness                            int64
    Online boarding                        int64
    Departure Delay in Minutes             int64
    Arrival Delay in Minutes             float64
    dtype: object

**Question:** Identify the target (or predicted) variable for passenger
satisfaction. What is your initial hypothesis about which variables will
be valuable in predicting satisfaction?

-   **satisfaction** represents the classification variable to be
    predicted.
-   Many of these variables seem like meaningful predictors pf
    satisfactio. In particular, delays correlated with satisfaction.

## Step 2: Model preparation<a href="#Step-2:-Model-preparation" class="anchor-link">¶</a>

Before you proceed with modeling, consider which metrics you will
ultimately want to leverage to evaluate your model.

**Question:** Which metrics are most suited to evaluating this type of
model?

-   As this is a binary classification probelm, it will be important to
    evaluate not just accuracy, but the balance of false positives and
    false negatives that the model's predictions provide.
-   The ROC AUC (Area Under the Receiver Operating Characeterisitc)
    score is also suited to this type of modeling.

### Prepare your data for predictions<a href="#Prepare-your-data-for-predictions" class="anchor-link">¶</a>

You may have noticed when previewing your data that there are several
non-numerical variables (`object` data types) within the dataset.

To prepare this DataFrame for modeling, first convert these variables
into a numerical format.

In \[5\]:

    # Convert the object predictor variables to numerical dummies.
    airline_data_dummies = pd.get_dummies(airline_data,
                                              columns=['satisfaction', 'Customer Type', 'Type of Travel', 'Class'])

### Isolate your target and predictor variables<a href="#Isolate-your-target-and-predictor-variables"
class="anchor-link">¶</a>

Separately define the target variable (`satisfaction`) and the features.

In \[6\]:

    # Define the y (target) variable.
    y = airline_data_dummies['satisfaction_satisfied']

    # Define the X (predictor) variables.
    X = airline_data_dummies.drop(['satisfaction_satisfied', 'satisfaction_dissatisfied'], axis = 1)

### Divide your data<a href="#Divide-your-data" class="anchor-link">¶</a>

Divide your data into a training set (75% of the data) and test set (25%
of the data). This is an important step in the process, as it allows you
to reserve a part of the data that the model has not used to test how
well the model generalizes (or performs) on new data.

In \[7\]:

    # Perform the split operation on your data.
    # Assign the outputs as follows: X_train, X_test, y_train, y_test.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Step 3: Model building<a href="#Step-3:-Model-building" class="anchor-link">¶</a>

### "Instantiate" your XGBClassifer<a href="#%22Instantiate%22-your-XGBClassifer" class="anchor-link">¶</a>

Before you fit your model to your airline dataset, first create the XGB
Classifier model and define its objective. You'll use this model to fit
and score different hyperparameters during the GridSearch
cross-validation process.

In \[8\]:

    # Define xgb to be your XGBClassifier.
    xgb = XGBClassifier(objective='binary:logistic', random_state=0)

### Define the parameters for hyperparameter tuning<a href="#Define-the-parameters-for-hyperparameter-tuning"
class="anchor-link">¶</a>

To identify suitable parameters for your `xgboost` model, first define
the parameters for hyperparameter tuning. Specifically, consider tuning
`max_depth`, `min_child_weight`, `learning_rate`, `n_estimators`,
`subsample`, and/or `colsample_bytree`.

Consider a more limited range for each hyperparameter to allow for
timely iteration and model training. For example, using a single
possible value for each of the six hyperparameters listed above will
take approximately one minute to run on this platform.

    {
        'max_depth': [4],
        'min_child_weight': [3],
        'learning_rate': [0.1],
        'n_estimators': [5],
        'subsample': [0.7],
        'colsample_bytree': [0.7]
    }

If you add just one new option, for example by changing `max_depth: [4]`
to `max_depth: [3, 6]`, and keep everything else the same, you can
expect the run time to approximately double. If you use two
possibilities for each hyperparameter, the run time would extend to \~1
hour.

In \[9\]:

    # Define parameters for tuning as `cv_params`.
    cv_params = {'max_depth': [4, 6],
                   'min_child_weight': [3, 5],
                   'learning_rate': [0.1, 0.2, 0.3],
                   'n_estimators': [5,10,15],
                   'subsample': [0.7],
                   'colsample_bytree': [0.7]
                   }

**Question:** What is the likely effect of adding more estimators to
your GridSearch?

More estimators will initially improve the model's performance. However,
increasing the number of estimators will also considerably increase the
time spent during the GridSearch process, and there will be diminishing
returns as the number of estimators continues to increase.

### Define how the models will be evaluated<a href="#Define-how-the-models-will-be-evaluated"
class="anchor-link">¶</a>

Define how the models will be evaluated for hyperparameter tuning. To
yield the best understanding of model performance, utilize a suite of
metrics.

In \[10\]:

    # Define your criteria as `scoring`.
    scoring = {'accuracy', 'precision', 'recall', 'f1'}

### Construct the GridSearch cross-validation<a href="#Construct-the-GridSearch-cross-validation"
class="anchor-link">¶</a>

Construct the GridSearch cross-validation using the model, parameters,
and scoring metrics you defined. Additionally, define the number of
folds and specify *which metric* from above will guide the refit
strategy.

In \[11\]:

    # Construct your GridSearch.
    xgb_cv = GridSearchCV(xgb, cv_params, scoring = scoring, cv=5, refit='f1')

### Fit the GridSearch model to your training data<a href="#Fit-the-GridSearch-model-to-your-training-data"
class="anchor-link">¶</a>

If your GridSearch takes too long, revisit the parameter ranges above
and consider narrowing the range and reducing the number of estimators.

**Note:** The following cell might take several minutes to run.

In \[ \]:

    %%time
    # fit the GridSearch model to training data
    xgb_cv = xgb_cv.fit(X_train, y_train)
    xgb_cv

**Question:** Which optimal set of parameters did the GridSearch yield?

Through accessing the **best*params*** attribute of the fitted
GridSearch model, the optimal set of hyperparameters was:
{'colsample_bytree': 0.7, 'learning_rate': 0.3, 'max_depth': 6,
'min_child_weight': 5, 'n_estimators': 15, 'subsample': 0.7}

### Save your model for reference using `pickle`<a href="#Save-your-model-for-reference-using-pickle"
class="anchor-link">¶</a>

Use the `pickle` library you've already imported to save the output of
this model.

In \[ \]:

    # Use `pickle` to save the trained model.
    pickle.dump(xbg_cv, open('xgb_cv.sav', 'wb'))

## Step 4: Results and evaluation<a href="#Step-4:-Results-and-evaluation" class="anchor-link">¶</a>

### Formulate predictions on your test set<a href="#Formulate-predictions-on-your-test-set"
class="anchor-link">¶</a>

To evaluate the predictions yielded from your model, leverage a series
of metrics and evaluation techniques from scikit-learn by examining the
actual observed values in the test set relative to your model's
prediction.

First, use your trained model to formulate predictions on your test set.

In \[ \]:

    # Apply your model to predict on your test data. Call this output "y_pred".
    y_pred = xgb_cv.predict(X_test)

### Leverage metrics to evaluate your model's performance<a href="#Leverage-metrics-to-evaluate-your-model&#39;s-performance"
class="anchor-link">¶</a>

Apply a series of metrics from scikit-learn to assess your model.
Specifically, print the accuracy score, precision score, recall score,
and f1 score associated with your test data and predicted values.

In \[ \]:

    # 1. Print your accuracy score.
    ac_score = metrics.accuracy_score(y_test, y_pred)
    print('accuracy score:', ac_score)

    # 2. Print your precision score.
    pc_score = metrics.precision_score(y_test, y_pred)
    print('precision score:', pc_score)

    # 3. Print your recall score.
    rc_score = metrics.recall_score(y_test, y_pred)
    print('recall score:', rc_score)

    # 4. Print your f1 score.
    f1_score = metrics.f1_score(y_test, y_pred)
    print('f1 score:', f1_score)

**Question:** How should you interpret your accuracy score?

The accuracy score for this model is 0.939, or 93.9% accurate.

**Question:** Is your accuracy score alone sufficient to evaluate your
model?

In classification problems, accuracy is useful to know but may not be
the best metric to evaluate this model.

**Question:** When observing the precision and recall scores of your
model, how do you interpret these values, and is one more accurate than
the other?

Precision and recall scores are both useful to evaluate the correct
predictive capability of the model because they balance the false
positives and false negatives inherent in prediction. The model shows a
precision score of 0.948, suggesting the model is very good at
predicting true positives. This means the model correctly predicts
whether the airline passenger will be satisfied. The recall score of
0.940 is also very good. This means that the model does a good job of
correctly identifying dissatisfied passengers within the dataset. These
two metrics combined give a better assessment of model performance than
the accuracy metric does alone.

**Question:** What does your model's F1 score tell you, beyond what the
other metrics provide?\*

The F1 score balances the precision and recall performance to give a
combined assessment of how well this model delivers predictions. The F1
score is 0.944, which suggests very strong predictive power in this
model.

### Gain clarity with the confusion matrix<a href="#Gain-clarity-with-the-confusion-matrix"
class="anchor-link">¶</a>

Recall that a **confusion matrix** is a graphic that shows a model's
true and false positives and true and false negatives. It helps to
create a visual representation of the components feeding into the
metrics above.

Create a confusion matrix based on your predicted values for the test
set.

In \[ \]:

    # Construct and display your confusion matrix.

    # Construct the confusion matrix for your predicted and test values.
    cm = metrics.confusion_matrix(y_tes, y_pred)

    # Create the display for your confusion matrix.
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb_cv.classes_)

    # Plot the visual in-line.
    disp.plot()

**Question:** When observing your confusion matrix, what do you notice?
Does this correlate to any of your other calculations?

The top left to bottom right diagonal in the confusion matrix represents
the correct predictions, and the ratio of these squares showcases the
accuracy.

The concentration of true positives and trie negatives stands out
relative to false positives and false negatives, respectively. This
ratio is why the precision score is so high 0.944.

### Visualize most important features<a href="#Visualize-most-important-features" class="anchor-link">¶</a>

`xgboost` has a built-in function to visualize the relative importance
of the features in the model using `matplotlib`. Output and examine the
feature importance of your model.

In \[ \]:

    # Plot the relative feature importance of the predictor variables in your model.
    plot_importance(xgb_cv.best_estimator_)

**Question:** Examine the feature importances outputted above. What is
your assessment of the result? Did anything surprise you?

-   By a wide margin, **seat comfort** rated as most important in the
    model. The type of seating is very different between first class and
    coach seating. However, the perks of being in first class also go
    beyond the seating type. Perhaps that is an underlying explanation
    of this feature#s importance.
-   Surprisingly, delays did not score as highly important.

### Compare models<a href="#Compare-models" class="anchor-link">¶</a>

Create a table of results to compare model performance.

In \[ \]:

    # Create a table of results to compare model performance.
    table = pd.DataFrame({'Model': ["Tuned Decision Tree", "Tuned Random Forest", "Tuned XGBoost"],
                          'F1': [0.945422, 0.947306, f1_score],
                          'Recall': [0.935863, 0.944501, rc_score],
                          'Precision': [0.955197, 0.950128, pc_score],
                          'Accuracy': [0.940864, 0.942450, ac_score]
                         }
                        )
    table

**Question:** How does this model compare to the decision tree and
random forest models you built in previous labs?

Based on the results shown in the table above, the F1, precision,
recall, and accuracy scores of the XGBoost model are similar to the
corresponding scores of the decision tree and random forest models. The
random forest model seemed to outperform the decision tree model as well
as the XGBoost model.

## Considerations<a href="#Considerations" class="anchor-link">¶</a>

**What are some key takeaways you learned from this lab?**

-   The evaluation of the model is important to inform if the model has
    delivered accurate predictions.
-   Splitting the data is important for ensuring that there is new data
    for the model to test its predictive performance.
-   Each metric provides an evaluation from a different standpoint, and
    accuracy alone is not a strong way to evaluate a model.
-   Effective assessments balance the true/false positives versus
    true/false negatives through the confusion matrix and F1 score.

**How would you share your findings with your team?**

-   Showcase the data used to create the prediction and the performance
    of the model overall.
-   Review the sample output of the features and the confusion matrix to
    reference the model's performance.
-   Highlight the metric values, emphasizing the F1 score.
-   Visualize the feature importance to showcase what drove the model's
    predictions.

**What would you share with and recommend to stakeholders?**

-   The model created is highly effective at predicting passenger
    satisfaction.
-   The feature importance of seat comfort warrants additional
    investigation. It will be important to ask domain experts why they
    believe this feature scores so highly in this model.

**Congratulations!** You've completed this lab. However, you may not
notice a green check mark next to this item on Coursera's platform.
Please continue your progress regardless of the check mark. Just click
on the "save" icon at the top of this notebook to ensure your work has
been logged